### Day 10 Problem

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

- If a chunk opens with (, it must close with ). 
- If a chunk opens with [, it must close with ].
- If a chunk opens with {, it must close with }.
- If a chunk opens with <, it must close with >.

So, () is a legal chunk that contains no other chunks, as is []. More complex but valid chunks include ([]), {()()()}, <([{}])>, [<>({}){}[([])<>]], and even (((((((((()))))))))).

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

## Proposed Approach

How I will solves this problem:
- If a close value occurs, check how many openers of same value happen before. Number of openers must >= number of closers
- If a close happens, find the closest opener and make sure everything inside is opened and closed
- If everything inside is opened and closed then that is a complete chunk otherwise corrupt

In [2]:
#import libraries
import pandas as pd
import numpy as np
import re
import requests

In [229]:
#loaded in the example given for testing on smaller sample size
test = pd.read_csv('day10test.csv')
test

,Strings
0,[({(<(())[]>[[{[]{<()<>>
1,[(()[<>])]({[<{<<[]>>(
2,{([(<{}[<>[]}>{[]{[(<()>
3,(((({<>}<{<{<>}{[]{[]{}
4,[[<[([]))<([[{}[[()]]]
5,[{[{({}]{}}([{[{{{}}([]
6,{<[[]]>}<{[{[{[]{()[[[]
7,[<(<(<(<{}))><([]([]()
8,<{([([[(<>()){}]>(<<{{
9,<{([{{}}[<[[[<>{}]]]>[]]


In [79]:
test['Strings'][0]

'[({(<(())[]>[[{[]{<()<>>'

In [606]:
###Formulas###
def point_counter(error):
    #this formula takes in the incorrect closing value of a supposed
    #closed chunk and assigns points based on the value
    points = 0
    if error == ')':
        points += 3
    elif error == ']':
        points += 57
    elif error == '}':
        points += 1197
    elif error == '>':
        points += 25137
    return points

def create_close_pos_df(currentline):
#returns a dataframe that holds the closing indicator we are looking for along with position of line it falls in
    pattern = re.compile(r'[>)}\]]')
    matches = pattern.finditer(currentline)
    count = 0
    #need a way to get a range of how many closers we had
    for i in matches:
        count+=1
    closer_df = pd.DataFrame(index = range(count), columns=['Value', 'Start Pos', 'End Pos'])
    z=0
    #need to recall compile and matches to be able to iterate again
    pattern = re.compile(r'[>)}\]]')
    matches = pattern.finditer(currentline)
    for match in matches:
        closer_df['Value'][z] = match[0]
        y = match.span()
        start_ind = y[0]
        closer_df['Start Pos'][z] = start_ind
        end_ind = y[1]
        closer_df['End Pos'][z] = end_ind
        z+=1
    return closer_df

def runner(df):
#Runs all code to return the total numeric value based on the number of corrupted lines in the dataset

    point_list = list()
    runner_ind = 0
    while runner_ind < len(df):
        #loop through each line in the input

        currentline = df[runner_ind]
        z = create_close_pos_df(currentline)
        text = ''
        z_index = 0
        range_list = list(range(len(currentline)))
        #create a list based on length which helps us handle when we have instances such as (()). We want to skip
        #over the first close when we look at the outer paranthesis
        while z_index < len(z): 
            #start with the first closer value in the text string. Any of these: >)]}
            curr_close_pos = z.loc[z_index]['Start Pos']
            curr_close_val = z.loc[z_index]['Value']
    #         print(curr_close_pos)
            curr_open_val = ''
            curr_open_pos = 0
            #Now we need to find the nearest open value for the given close. If close is ) then whats nearest (?
            if curr_close_val == '}':
                curr_open_val = '{'
            elif curr_close_val == ']':
                curr_open_val = '['
            elif curr_close_val == '>':
                curr_open_val = '<'
            elif curr_close_val == ')':
                curr_open_val = '('

            b = curr_close_pos
            while b > 0:
                #Find index of the nearest opener
                if (currentline[b] == curr_open_val) and (b in range_list):
                    #needed to add 2nd argument to skip if opener already
                    #matched to another close. Instances of (())
                    curr_open_pos = b
                    range_list.remove(curr_open_pos)
                    break
                b-=1
        #     print(b)
    #         print(curr_open_pos)
            # print(curr_close_pos)

            #returns a new string with just values inside close. Allows us to analyze inside the close whether items
            #have been incorrectly opened. If there are 2 (( but only 1 ) then this is corrupt
            newline = currentline[curr_open_pos:curr_close_pos+1]

            q = curr_close_pos-1
            while q > curr_open_pos:
        #         print(q)
                val = currentline[q]
            #     print(val)
                #gives us our pattern that we use to return counts in the current new string of openers vs closers
                if (val == ')') or (val == '('):
                    closer_pattern=re.compile(r'\)')
                    opener_pattern = re.compile(r'\(')
                elif (val == '>') or (val == '<'):
                    closer_pattern = re.compile(r'\>')
                    opener_pattern = re.compile(r'\<')
                elif (val == ']') or (val == '['):
                    closer_pattern = re.compile(r'\]')
                    opener_pattern = re.compile(r'\[')
                elif (val == '}') or (val == '{'):
                    closer_pattern = re.compile(r'\}')
                    opener_pattern = re.compile(r'\{')
                closer_matches = closer_pattern.findall(newline)
            #     print(closer_matches)
                closer_count = 0
                closer_count = len(closer_matches)
            #     print('closer count: ', closer_count)
            #         print('# of {} before or at current position: {}'.format(closer_val, closer_count))

                #return how many openers of that closing char came before current position
                opener_matches = opener_pattern.findall(newline)
                opener_count = 0
                opener_count = len(opener_matches)
            #     print('opener count: ', opener_count)
                if opener_count > closer_count:
                    #if we have more openers then closers then we have a problem which we mark as corrupt
                    #if it is corrupt we stop and break out the loop
                    text = 'corrupt'
                    break

                q-=1
            error = '' 
            points = 0
            #if we have a corrupt instance we want to know what indexed value was it that caused the problem
            #this is how we get the point value associated with that problem
            if text == 'corrupt':
                error = currentline[curr_close_pos]
                point_val = point_counter(error)
                print('point val: ', point_val)
                point_list.append(point_val)
                #Append to a list to make it easy to sum up all values
                break
            z_index +=1
        runner_ind+=1

    #sum up everything in list and return this value
    answer = sum(point_list)
    return answer




In [607]:
runner(test['Strings'])

point val:  1197
point val:  3
point val:  57
point val:  3
point val:  25137


26397

In [581]:
#26397 is expected answer, lets try on input data now

In [590]:
day10 = pd.read_csv("day_10.txt", header=None)

In [604]:
runner(day10[0])

point val:  57
point val:  1197
point val:  1197
point val:  25137
point val:  25137
point val:  3
point val:  57
point val:  25137
point val:  25137
point val:  57
point val:  3
point val:  57
point val:  1197
point val:  57
point val:  57
point val:  57
point val:  25137
point val:  1197
point val:  3
point val:  1197
point val:  3
point val:  3
point val:  57
point val:  57
point val:  25137
point val:  57
point val:  25137
point val:  1197
point val:  1197
point val:  1197
point val:  57
point val:  25137
point val:  25137
point val:  1197
point val:  3
point val:  57
point val:  1197
point val:  57
point val:  25137
point val:  1197
point val:  1197
point val:  1197
point val:  3
point val:  25137
point val:  57
point val:  25137
point val:  57


318081

In [595]:
#318,081 is the correct answer for part 1. Ready for part 2!